<a href="https://colab.research.google.com/github/VintageGold/Text_Summarization/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

%cd drive/My \ Drive/Text_Summarization

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My  Drive/Text_Summarization


In [2]:
import json
import pandas as pd

# Load Data

In [3]:
with open("Data/telehealth_article_texts.txt") as f:
    for i in range(4):
        next(f)
    corpus = f.read()

In [4]:
dict_articles = json.loads(corpus.replace('\n',''))

In [8]:
df_metadata = pd.read_excel('MetaData/Telehealth article key 2.25.xlsx',dtype={'Index':'object'},index_col='Index')

In [9]:
df_metadata['Content'] = dict_articles.values()

In [10]:
df_metadata

,Journal Title,Article Title,Date Published,Authors,Abstract,Keywords,Citation,Content
Index,,,,,,,,
0,Psychological Servies,The Effectiveness of Telepsychology With Veter...,2021,Michael J. McClellan; Richard Osbaldiston; Ron...,Veterans face a variety of stressors due to th...,"KEYWORDS:\n\ntelepsychology, meta-analysis, ve...","McClellan, M. J., Osbaldiston, R., Wu, R., Yea...",Veterans face a variety of stressors related t...
1,"Psychology, Public Policy, and Law",Making the Case for Videoconferencing and Remo...,2021,Milfred D. Dale; Desiree Smith,The COVID-19 pandemic and its requirements for...,"KEYWORDS:\n\nremote child custody evaluations,...","Dale, M. D., & Smith, D. (2021). Making the ca...",The impact of the COVID-19 pandemic on the chi...
2,"Psychology, Public Policy, and Law","Forensic E-Mental Health: Review, Research Pri...",2021,Lauren E. Kois; Jennifer Cox; Ashley T. Peck,Forensic e-mental health is an area of psychol...,"KEYWORDS:\n\nforensic e-mental health, telehea...","Kois, L. E., Cox, J., & Peck, A. T. (2021). Fo...","In November 2019, COVID-19—the infectious, hig..."
3,Training and Education in Professional Psychology,Moving Toward a New Era of Telepsychology in U...,2021,Allison L. Baier; Sarah Danzo,Many university training clinics are facing nu...,"KEYWORDS:\n\neducation and training, telepsych...","Baier, A. L., & Danzo, S. (2021). Moving towar...",Anxiety and depressive disorders are among the...
4,Practice Innovations,Navigating Changes in the Physical and Psychol...,2021,Liat Shklarski; Allison Abrams; Elana Bakst,The emergence of the Covid-19 pandemic at the ...,"KEYWORDS:\n\nremote psychotherapy, Covid-19, p...","Shklarski, L., Abrams, A., & Bakst, E. (2021)....","In mid-March, 2020, the authors—as well as the..."
5,American Psychologist,The COVID-19 Telepsychology Revolution: A Nati...,2021,Bradford S. Pierce; Paul B. Perrin; Carmen M. ...,The COVID-19 pandemic has altered mental healt...,"KEYWORDS:\n\ntelepsychology, COVID-19, telemed...","Pierce, B. S., Perrin, P. B., Tyler, C. M., Mc...","In early January of 2020, Chinese health autho..."
6,Psychological Services,Pre-COVID-19 Deterrents to Practicing With Vid...,2020,Bradford S. Pierce; Paul B. Perrin; Scott D. M...,Telepsychology has the potential to help menta...,"KEYWORDS:\n\ntelepsychology, telehealth, telem...","Pierce, B. S., Perrin, P. B., & McDonald, S. D...",Millions of people with a diagnosable mental d...
7,Canadian Psychology/Psychologie canadienne,"COVID-19 and Telemental Health: Benefits, Chal...",2020,Sheri Madigan; Nicole Racine; Jessica E. Cooke...,Although the medical impacts of COVID-19 are n...,"KEYWORDS:\n\ntelemental health, telepsychology...","Madigan, S., Racine, N., Cooke, J. E., & Korcz...",The primary focus of COVID-19 has been on its ...
8,Psychological Assessment,"Equivalence of Remote, Digital Administration ...",2020,A. Jordan Wright,Many children in need do not receive the psych...,"KEYWORDS:\n\npsychoeducational evaluation, tel...","Wright, A. J. (2020). Equivalence of remote, d...",There is a worldwide gap between the mental he...
